<a href="https://colab.research.google.com/github/cheung0/Question-Answering/blob/main/Question_Answering_on_Documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Information Retreival - LangChain
- Without Using OpenAI Embeddings
- Without OpenAI LLM

Two Applications:
- Text Documents
- Multiple PDF Files

### Get HUGGINGFACEHUB_API_KEY

In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hidden village"

**Intro**

How to ask questions on a document?
So maybe you're a:
1. College student and have a Biology final exam coming up and want to ask questions to a tutor but it's 3 AM.
2. High school student on vacation and want to know about Taiwan's beautiful waterfalls before you go visit.
3. Patient with psoriasis and have questions you want to ask your doctor about excimer laser therapy. 

In this example. 
Text -> NLP Model -> do cool stuff like asking questions which is what I’m doing today

In [ ]:
!pip install langchain
!pip install huggingface_hub
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 5.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 

### Download Text File

In [ ]:
#import requests

#url = "https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt"
#res = requests.get(url)
#file_path = '/content/excimer laser.txt'  # Specify the desired file path
#with open(file_path, "w") as f:
#    f.write(res.text)

In [ ]:
# Document Loader
# Remember to change path!
path = '/Untitled document.txt'

# Read the contents of the input file
with open(path, 'r') as file:
    content = file.read()

# Split the content into individual sentences
sentences = content.split('. ')

# Remove leading and trailing spaces from each sentence
sentences = [sentence.strip() for sentence in sentences]

# Join the sentences with the desired format
formatted_text = ".\n\n".join(sentences)

# Write the formatted text to a new file
with open(path, 'w') as file:
    file.write(formatted_text)


from langchain.document_loaders import TextLoader
loader = TextLoader(path)

documents = loader.load()

In [ ]:
documents

[Document(page_content='\ufeffExcimer laser for the treatment of psoriasis: safety, efficacy, and patient acceptability.\n\nThe 308 nm excimer laser is a widely used device throughout the field of dermatology for many diseases including psoriasis.\n\nAlthough the laser has demonstrated clinical efficacy, there is a lack of literature outlining the safety, efficacy, and patient acceptability of the excimer laser.\nA literature search on PubMed was used with combinations of the terms “excimer”, “excimer laser”, “308 nm”, “psoriasis”, “protocol”, “safety”, “efficacy”, acceptability”, “side effects”, and “dose”.\n\nThe search results were included if they contained information pertaining to excimer laser and psoriasis treatment and description of the safety, efficacy, and patient acceptability of the treatment.\nThe 308 nm excimer laser is generally safe and well tolerated with minimal side effects including erythema, blistering, and pigmentary changes.\n\nIt has a range of efficacies depe

In [ ]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
print(wrap_text_preserve_newlines(str(documents[0])))

page_content='\ufeffExcimer laser for the treatment of psoriasis: safety, efficacy, and patient
acceptability.\n\nThe 308 nm excimer laser is a widely used device throughout the field of dermatology for
many diseases including psoriasis.\n\nAlthough the laser has demonstrated clinical efficacy, there is a lack
of literature outlining the safety, efficacy, and patient acceptability of the excimer laser.\nA literature
search on PubMed was used with combinations of the terms “excimer”, “excimer laser”, “308 nm”, “psoriasis”,
“protocol”, “safety”, “efficacy”, acceptability”, “side effects”, and “dose”.\n\nThe search results were
included if they contained information pertaining to excimer laser and psoriasis treatment and description of
the safety, efficacy, and patient acceptability of the treatment.\nThe 308 nm excimer laser is generally safe
and well tolerated with minimal side effects including erythema, blistering, and pigmentary changes.\n\nIt has
a range of efficacies depending on t

In [ ]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)

In [ ]:
len(docs)

35

In [ ]:
docs[0]

Document(page_content='\ufeffExcimer laser for the treatment of psoriasis: safety, efficacy, and patient acceptability.\n\nThe 308 nm excimer laser is a widely used device throughout the field of dermatology for many diseases including psoriasis.\n\nAlthough the laser has demonstrated clinical efficacy, there is a lack of literature outlining the safety, efficacy, and patient acceptability of the excimer laser.\nA literature search on PubMed was used with combinations of the terms “excimer”, “excimer laser”, “308 nm”, “psoriasis”, “protocol”, “safety”, “efficacy”, acceptability”, “side effects”, and “dose”.\n\nThe search results were included if they contained information pertaining to excimer laser and psoriasis treatment and description of the safety, efficacy, and patient acceptability of the treatment.\nThe 308 nm excimer laser is generally safe and well tolerated with minimal side effects including erythema, blistering, and pigmentary changes.', metadata={'source': '/Untitled docu

### Embeddings

In [ ]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [ ]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [ ]:
query = "Cost of using excimer laser therapy?"

"""
Queries 
query = "What is the mechanism of action of the excimer laser in treating psoriasis?"
query = "What are the potential benefits of using the excimer laser for psoriasis treatment?"
query = "Is excimer laser therapy safe and what are the side effects?"

query = "Cost of using excimer laser therapy?"
"""
docs = db.similarity_search(query)

In [ ]:
print(wrap_text_preserve_newlines(str(docs[0].page_content)))

﻿Excimer laser for the treatment of psoriasis: safety, efficacy, and patient acceptability.

The 308 nm excimer laser is a widely used device throughout the field of dermatology for many diseases
including psoriasis.

Although the laser has demonstrated clinical efficacy, there is a lack of literature outlining the safety,
efficacy, and patient acceptability of the excimer laser.
A literature search on PubMed was used with combinations of the terms “excimer”, “excimer laser”, “308 nm”,
“psoriasis”, “protocol”, “safety”, “efficacy”, acceptability”, “side effects”, and “dose”.

The search results were included if they contained information pertaining to excimer laser and psoriasis
treatment and description of the safety, efficacy, and patient acceptability of the treatment.
The 308 nm excimer laser is generally safe and well tolerated with minimal side effects including erythema,
blistering, and pigmentary changes.


In [ ]:
db.similarity_search_with_relevance_scores(query)

[(Document(page_content='\ufeffExcimer laser for the treatment of psoriasis: safety, efficacy, and patient acceptability.\n\nThe 308 nm excimer laser is a widely used device throughout the field of dermatology for many diseases including psoriasis.\n\nAlthough the laser has demonstrated clinical efficacy, there is a lack of literature outlining the safety, efficacy, and patient acceptability of the excimer laser.\nA literature search on PubMed was used with combinations of the terms “excimer”, “excimer laser”, “308 nm”, “psoriasis”, “protocol”, “safety”, “efficacy”, acceptability”, “side effects”, and “dose”.\n\nThe search results were included if they contained information pertaining to excimer laser and psoriasis treatment and description of the safety, efficacy, and patient acceptability of the treatment.\nThe 308 nm excimer laser is generally safe and well tolerated with minimal side effects including erythema, blistering, and pigmentary changes.', metadata={'source': '/Untitled do